In [17]:
!pip install opencv-python

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
import json
import cv2
import os
import numpy as np

class HandGestureDataset(Dataset):
    def __init__(self, json_file, img_dir, transform=None):
        with open(json_file, 'r') as f:
            self.annotations = json.load(f)
        self.img_dir = img_dir
        self.transform = transform
        self.classes = list(set(item['gesture'] for item in self.annotations.values()))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = list(self.annotations.keys())[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        annotation = self.annotations[img_name]
        gesture = annotation['gesture']
        bbox = annotation['bbox']

        label = self.class_to_idx[gesture]
        bbox = torch.tensor(bbox, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label, bbox

In [19]:
from torchvision import transforms
from torch.utils.data import random_split

# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset
dataset = HandGestureDataset('hand_gesture_data/augmented_annotations.json', 'hand_gesture_data', transform=transform)

# Split dataset into train and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [20]:
import torch.nn as nn
import torchvision.models as models

class HandGestureModel(nn.Module):
    def __init__(self, num_classes):
        super(HandGestureModel, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        self.fc1 = nn.Linear(num_features, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.fc3 = nn.Linear(128, 4)  # for bounding box regression

    def forward(self, x):
        features = self.resnet(x)
        features = torch.relu(self.fc1(features))
        class_output = self.fc2(features)
        bbox_output = self.fc3(features)
        return class_output, bbox_output

model = HandGestureModel(num_classes=len(dataset.classes))

In [ ]:
import torch.optim as optim

# Define loss functions and optimizer
criterion_cls = nn.CrossEntropyLoss()
criterion_bbox = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
device = torch.device("mps")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels, bboxes in train_loader:
        images, labels, bboxes = images.to(device), labels.to(device), bboxes.to(device)
        
        optimizer.zero_grad()
        class_outputs, bbox_outputs = model(images)
        loss_cls = criterion_cls(class_outputs, labels)
        loss_bbox = criterion_bbox(bbox_outputs, bboxes)
        loss = loss_cls + loss_bbox
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels, bboxes in val_loader:
            images, labels, bboxes = images.to(device), labels.to(device), bboxes.to(device)
            class_outputs, bbox_outputs = model(images)
            loss_cls = criterion_cls(class_outputs, labels)
            loss_bbox = criterion_bbox(bbox_outputs, bboxes)
            loss = loss_cls + loss_bbox
            val_loss += loss.item()
            _, predicted = class_outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, '
          f'Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {100.*correct/total:.2f}%')

# Save the model
torch.save(model.state_dict(), 'hand_gesture_model.pth')

Epoch 1/50, Train Loss: 322737.5295, Val Loss: 309184.2773, Val Acc: 31.25%
Epoch 2/50, Train Loss: 288092.0582, Val Loss: 258677.0039, Val Acc: 18.75%
Epoch 3/50, Train Loss: 214690.9592, Val Loss: 168053.3867, Val Acc: 43.75%
Epoch 4/50, Train Loss: 121329.3099, Val Loss: 72595.4902, Val Acc: 31.25%
Epoch 5/50, Train Loss: 65421.3485, Val Loss: 55393.1113, Val Acc: 31.25%
Epoch 6/50, Train Loss: 54744.6304, Val Loss: 66017.6914, Val Acc: 18.75%
Epoch 7/50, Train Loss: 54232.6587, Val Loss: 53090.4839, Val Acc: 31.25%
Epoch 8/50, Train Loss: 52706.7680, Val Loss: 53972.6699, Val Acc: 18.75%
Epoch 9/50, Train Loss: 51679.2747, Val Loss: 49044.0352, Val Acc: 25.00%
Epoch 10/50, Train Loss: 50396.9794, Val Loss: 51540.1655, Val Acc: 31.25%
Epoch 11/50, Train Loss: 49324.1430, Val Loss: 47987.8833, Val Acc: 31.25%
Epoch 12/50, Train Loss: 46401.5046, Val Loss: 43862.0366, Val Acc: 18.75%
Epoch 13/50, Train Loss: 42978.9569, Val Loss: 37859.8203, Val Acc: 12.50%
Epoch 14/50, Train Loss: 40